<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/Limpieza_SO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Limipieza SOCIOS

In [66]:
# from google.colab import drive
# drive.mount('/content/drive')

KeyboardInterrupt: 

In [ ]:
pip install unidecode

In [67]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

In [68]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
CU_SO = pd.read_excel('/content/CU_SO.xlsx')
TO_SO = pd.read_excel('/content/TO_SO.xlsx')
AB_SO = pd.read_excel('/content/AB_SO.xlsx')
GU_SO = pd.read_excel('/content/GU_SO.xlsx')
CR_SO = pd.read_excel('/content/CR_SO.xlsx')

In [69]:
# # Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
# folder_path = '/content/drive/My Drive/PFG_FASPAS/SO'
# docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
# print(docs_xlsx)

In [70]:
# dic_dataframes = {}

# for doc in docs_xlsx:
#     entire_path = os.path.join(folder_path, doc)
#     df = pd.read_excel(entire_path)
#     # Uso el nombre del archivo como clave
#     dic_dataframes[doc] = df

In [71]:
# # Imprimir todas las claves
# for clave in dic_dataframes.keys():
#     print(clave)

In [72]:
# Definir la función que realiza las transformaciones
def transform_df(df):
    col_excluded = ['FECHA NACIMIENTO', 'FECHA DE NACIMIENTO', 'FECHA NAC']

    # Convertir todas las columnas de tipo object a mayúsculas, excepto las especificadas
    for column in df.columns:
        if df[column].dtype == 'object' and column not in col_excluded:
            df[column] = df[column].apply(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

    # Ahora, aplicamos unidecode a los nombres de las columnas
    df.columns = [unidecode(col.upper()) for col in df.columns]

    return df

def mapping_names(df):
    # Diccionario con los mapeos deseados
    columns_map = {
        'CODIGO POSTAL': 'CP',
        'MUNICIPIO': 'LOCALIDAD',
        'FECHA DE SOCIO/A': 'FECHA ALTA SOCIO',
        'FECHA INSCRIPCION': 'FECHA ALTA SOCIO',
        'ANO SOCIO': 'YEAR FECHA ALTA SOCIO',
        'FECHA NAC': 'FECHA NACIMIENTO',
        'F NAC': 'FECHA NACIMIENTO',
        'PROT AUD': 'PROTESIS',
        'SEXO': 'GENERO'
    }

    # Crear un nuevo diccionario para los nombres de columnas
    rename_columns = {}

    # Iterar sobre las columnas y aplicar el mapeo
    for col in df.columns:
        norm_col = columns_map.get(col, col)
        rename_columns[col] = norm_col

    # Renombrar las columnas del DataFrame
    df.rename(columns=rename_columns, inplace=True)
    return df

def AB_localidad_CP(df):
  # Extraer el código postal (CP)
  df['CP'] = df['LOCALIDAD'].str.extract(r'(\d+)', expand=False)

  # Extraer el nombre de la localidad (LOCALIDAD)
  df['LOCALIDAD'] = df['LOCALIDAD'].str.extract(r'(\D+)', expand=False).str.strip()

  if 'FECHA ALTA SOCIO' in df.columns:
    print('ENTRA')
    df['YEAR FECHA ALTA SOCIO'] = df['FECHA ALTA SOCIO'].dt.year
  return df

def CP_year(df):
    if 'FECHA NACIMIENTO' in df.columns:
        if not pd.api.types.is_datetime64_any_dtype(df['FECHA NACIMIENTO']):
            df['FECHA NACIMIENTO'] = pd.to_datetime(df['FECHA NACIMIENTO'], errors='coerce')

        df['FECHA NACIMIENTO'] = df['FECHA NACIMIENTO'].apply(lambda x: np.nan if str(x).isdigit() or x == '(ADULTO)' else x)
        df['YEAR NACIMIENTO'] = df['FECHA NACIMIENTO'].dt.year

    if 'CP' in df.columns:
        # Convertir a numérico, dejando como NaN los valores que no sean convertibles
        df['CP'] = pd.to_numeric(df['CP'], errors='coerce').astype('Int64')

        df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')
        CP_loc_filter = df['CP'].isnull() & df['LOCALIDAD'].isnull()
        df = df[~CP_loc_filter]

    elif 'LOCALIDAD' in df.columns:
        df = df.dropna(subset=['LOCALIDAD'], how='all')
        loc_filter = df['LOCALIDAD'].isnull()
        df = df[~loc_filter]

    # Limpiar los datos eliminando contenido entre paréntesis
    df['LOCALIDAD'] = df['LOCALIDAD'].str.replace(r'\s*\([^)]*\)', '', regex=True)

    if 'FECHA ALTA SOCIO' in df.columns:
      # Ensure 'FECHA ALTA SOCIO' is in datetime format
      df['FECHA ALTA SOCIO'] = pd.to_datetime(df['FECHA ALTA SOCIO'], errors='coerce')
      # Create 'YEAR ALTA SOCIO' column
      df['YEAR ALTA SOCIO'] = df['FECHA ALTA SOCIO'].dt.year
    return df

def keep_columns(df):
    desired_columns = ['FECHA NACIMIENTO', 'GENERO', 'LOCALIDAD', 'CP', 'PROVINCIA', 'YEAR NACIMIENTO', 'FECHA ALTA SOCIO', 'YEAR ALTA SOCIO']
    # Filtrar el DataFrame para incluir solo las columnas que existen en el DataFrame
    filtered_df = df[df.columns.intersection(desired_columns)]


    if 'GENERO' in df.columns:
      # Normalize the 'GENERO' column
      mujer = ['MUJER', 'HEMBRA', 'FEMENINO', 'FEM', 'F', 'M']
      hombre = ['HOMBRE', 'MACHO', 'MASCULINO', 'MASC', 'MAS', 'H']

      filtered_df['GENERO'] = filtered_df['GENERO'].replace(mujer, 'MUJER')
      filtered_df['GENERO'] = filtered_df['GENERO'].replace(hombre, 'HOMBRE')

    return filtered_df

def delete_rows(df):
  if 'CP' in df.columns:
    df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')
  else:
    df = df.dropna(subset=['LOCALIDAD'], how='all')
  return df

In [73]:
# for df_key, df in dic_dataframes.items():
#     # Apply transformations for DataFrames with keys containing 'AB'
#     if 'AB' in df_key:
#         dic_dataframes[df_key] = transform_df(df)
#         dic_dataframes[df_key] = mapping_names(df)
#         dic_dataframes[df_key] = AB_localidad_CP(df)
#         dic_dataframes[df_key] = keep_columns(df)
#         dic_dataframes[df_key] = delete_rows(df)
#     else:
#         # Apply different transformations for other DataFrames
#         dic_dataframes[df_key] = transform_df(df)
#         dic_dataframes[df_key] = mapping_names(df)
#         dic_dataframes[df_key] = CP_year(df)
#         dic_dataframes[df_key] = keep_columns(df)
#         dic_dataframes[df_key] = delete_rows(df)

In [74]:
# CÓDIGO PARA EL CURRO PQ NO PUEDO USAR EL DRIVE
AB_SO = transform_df(AB_SO)
AB_SO = mapping_names(AB_SO)
AB_SO = AB_localidad_CP(AB_SO)
AB_SO = CP_year(AB_SO)
AB_SO = keep_columns(AB_SO)
AB_SO = delete_rows(AB_SO)

CU_SO = transform_df(CU_SO)
CU_SO = mapping_names(CU_SO)
CU_SO = CP_year(CU_SO)
CU_SO = keep_columns(CU_SO)
CU_SO = delete_rows(CU_SO)

CR_SO = transform_df(CR_SO)
CR_SO = mapping_names(CR_SO)
CR_SO = CP_year(CR_SO)
CR_SO = keep_columns(CR_SO)
CR_SO = delete_rows(CR_SO)

GU_SO = transform_df(GU_SO)
GU_SO = mapping_names(GU_SO)
GU_SO = CP_year(GU_SO)
GU_SO = keep_columns(GU_SO)
GU_SO = delete_rows(GU_SO)

TO_SO = transform_df(TO_SO)
TO_SO = mapping_names(TO_SO)
TO_SO = CP_year(TO_SO)
TO_SO = keep_columns(TO_SO)
TO_SO = delete_rows(TO_SO)

ENTRA


<ipython-input-72-564f3aaa2947>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['GENERO'] = filtered_df['GENERO'].replace(mujer, 'MUJER')
<ipython-input-72-564f3aaa2947>:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['GENERO'] = filtered_df['GENERO'].replace(hombre, 'HOMBRE')
<ipython-input-72-564f3aaa2947>:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

KeyError: ['CP']

In [ ]:
CLM_SO = pd.concat([AB_SO, CR_SO, CU_SO, GU_SO, TO_SO])

In [ ]:
# List of DataFrames
dfs = [AB_SO, CR_SO, CU_SO, GU_SO, TO_SO, CLM_SO]

# Corresponding folder paths on Google Drive
folder_paths = [
    '/content/drive/My Drive/PFG_FASPAS/AB',
    '/content/drive/My Drive/PFG_FASPAS/CR',
    '/content/drive/My Drive/PFG_FASPAS/CU',
    '/content/drive/My Drive/PFG_FASPAS/GU',
    '/content/drive/My Drive/PFG_FASPAS/TO',
    '/content/drive/My Drive/PFG_FASPAS/CLM'
]

# Corresponding file names
file_names = ['AB_SO_limpio.xlsx', 'CR_SO_limpio.xlsx', 'CU_SO_limpio.xlsx', 'GU_SO_limpio.xlsx', 'TO_SO_limpio.xlsx', 'CLM_SO_limpio.xlsx']

In [ ]:
# # Iterate over the DataFrames, folder paths, and file names
# for df, folder_path, file_name in zip(dfs, folder_paths, file_names):
#     # Define the complete file path
#     file_path = f"{folder_path}/{file_name}"

#     # Save the DataFrame as an Excel file
#     df.to_excel(file_path, index=False)

# print("All DataFrames have been exported successfully.")

In [ ]:
CLM_SO